In [1]:
import pandas as pd 
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import plotly.express as px

In [3]:
incoming_df = pd.read_parquet("train/incoming_run_data_1.parquet")
incoming_df_filtered = incoming_df[incoming_df["Run ID"] == "6515456b-05d4-5e6d-b42a-7652a8d671a4"]
incoming_df_filtered

,Tool ID,Run Start Time,Run End Time,Run ID,Process Step,Step ID,Time Stamp,Sensor Name,Sensor Value
0,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2023-12-31,2023-12-31 00:12:22,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,Step_0,2023-12-31 00:00:00.000000000,Sensor_1,202.660553
1,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2023-12-31,2023-12-31 00:12:22,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,Step_0,2023-12-31 00:00:01.000000002,Sensor_1,202.660553
2,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2023-12-31,2023-12-31 00:12:22,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,Step_0,2023-12-31 00:00:01.999999997,Sensor_1,202.660553
3,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2023-12-31,2023-12-31 00:12:22,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,Step_1,2023-12-31 00:00:03.000000000,Sensor_1,202.660553
4,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2023-12-31,2023-12-31 00:12:22,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,Step_1,2023-12-31 00:00:04.000000002,Sensor_1,202.660553
...,...,...,...,...,...,...,...,...,...
21151,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2023-12-31,2023-12-31 00:12:22,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,Step_7,2023-12-31 00:08:31.000000002,Sensor_9,-0.263628
21152,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2023-12-31,2023-12-31 00:12:22,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,Step_7,2023-12-31 00:08:31.999999998,Sensor_9,2.485652
21153,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2023-12-31,2023-12-31 00:12:22,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,Step_7,2023-12-31 00:08:33.000000000,Sensor_9,0.828387
21154,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2023-12-31,2023-12-31 00:12:22,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,Step_7,2023-12-31 00:08:34.000000002,Sensor_9,1.820897


In [10]:
tmp_list = []

temp_grps = [(x[0], x[1], y) for x, y in incoming_df_filtered.groupby(by=["Step ID", "Sensor Name"])]
for temp_grp in temp_grps:
    temp_step_id = temp_grp[0]
    temp_sensor_name = temp_grp[1]
    temp_df = temp_grp[2]
    temp_df = temp_df.reset_index()

    temp_tool_id = temp_df.loc[0, "Tool ID"]
    temp_run_start_time = temp_df.loc[0, "Run Start Time"]
    temp_run_end_time = temp_df.loc[0, "Run End Time"]
    temp_run_id = temp_df.loc[0, "Run ID"]
    temp_process_step = temp_df.loc[0, "Process Step"]

    sensor_avgs = temp_df["Sensor Value"].mean()

    temp_time_taken = (temp_df["Time Stamp"].max() - temp_df["Time Stamp"].min()).total_seconds()
    temp_time_taken = int(round(temp_time_taken))
   
    tmp_list.append([
        temp_process_step,
        temp_tool_id,
        temp_run_id,
        temp_run_start_time,
        temp_run_end_time,
        temp_time_taken,
        temp_step_id,
        temp_sensor_name,
        sensor_avgs
    ]) 

columns = [
    "Process Step",
    "Tool ID",
    "Run ID",
    "Run Start Time",
    "Run End Time",
    "Time Taken (Seconds)",
    "Step ID",
    "Sensor Name",
    "Sensor Value (Average)"
] 
   
dataframe_incoming_filtered = pd.DataFrame(tmp_list, columns=columns) 
del temp_grps
dataframe_incoming_filtered = dataframe_incoming_filtered.sort_values(
    by=["Step ID", "Sensor Name"], 
    key=lambda x: x.str.extract(r'_(\d+)')[0].astype(int)
)
dataframe_incoming_filtered


,Process Step,Tool ID,Run ID,Run Start Time,Run End Time,Time Taken (Seconds),Step ID,Sensor Name,Sensor Value (Average)
0,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,2,Step_0,Sensor_1,202.660568
11,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,2,Step_0,Sensor_2,818.553528
22,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,2,Step_0,Sensor_3,19.557402
33,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,2,Step_0,Sensor_4,997.134766
36,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,2,Step_0,Sensor_5,3.115842
...,...,...,...,...,...,...,...,...,...
317,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,38,Step_7,Sensor_37,0.048179
318,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,38,Step_7,Sensor_38,51.704819
319,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,38,Step_7,Sensor_39,51.982334
321,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,38,Step_7,Sensor_40,6.482771


In [12]:
dataframe_incoming_filtered["Test"] = dataframe_incoming_filtered["Sensor Name"] + ";" + dataframe_incoming_filtered["Step ID"]
dataframe_incoming_filtered = dataframe_incoming_filtered.drop(columns=["Step ID", "Sensor Name"])
dataframe_incoming_filtered

,Process Step,Tool ID,Run ID,Run Start Time,Run End Time,Time Taken (Seconds),Sensor Value (Average),Test
0,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,2,202.660568,Sensor_1;Step_0
11,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,2,818.553528,Sensor_2;Step_0
22,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,2,19.557402,Sensor_3;Step_0
33,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,2,997.134766,Sensor_4;Step_0
36,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,2,3.115842,Sensor_5;Step_0
...,...,...,...,...,...,...,...,...
317,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,38,0.048179,Sensor_37;Step_7
318,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,38,51.704819,Sensor_38;Step_7
319,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,38,51.982334,Sensor_39;Step_7
321,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,38,6.482771,Sensor_40;Step_7


In [24]:
dataframe_incoming_pivot = dataframe_incoming_filtered.pivot(
    index=["Process Step", "Tool ID", "Run ID", "Run Start Time", "Run End Time"],
    columns="Test",
    values="Sensor Value (Average)"
).reset_index()

dataframe_incoming_pivot

Test,Process Step,Tool ID,Run ID,Run Start Time,Run End Time,Sensor_10;Step_0,Sensor_10;Step_1,Sensor_10;Step_2,Sensor_10;Step_3,Sensor_10;Step_4,...,Sensor_8;Step_6,Sensor_8;Step_7,Sensor_9;Step_0,Sensor_9;Step_1,Sensor_9;Step_2,Sensor_9;Step_3,Sensor_9;Step_4,Sensor_9;Step_5,Sensor_9;Step_6,Sensor_9;Step_7
0,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,2023-12-31,2023-12-31 00:12:22,2.087202,162.857635,178.414047,6.132487,1.754854,...,50.293026,52.600311,27.235899,129.909378,97.5998,113.357437,105.180557,100.762833,49.414692,52.082336


In [13]:
run_df = pd.read_parquet("train/run_data_1.parquet")
run_df_filtered = run_df[run_df["Run ID"] == "6515456b-05d4-5e6d-b42a-7652a8d671a4"]

run_df_filtered

,Tool ID,Run Start Time,Run End Time,Run ID,Process Step,Consumable Life,Step ID,Time Stamp,Sensor Name,Sensor Value
0,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2024-01-01,2024-01-01 00:12:35,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,370.7229,Step_0,2024-01-01 00:00:00.000000000,Sensor_A,-0.061105
1,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2024-01-01,2024-01-01 00:12:35,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,370.7229,Step_0,2024-01-01 00:00:01.000000002,Sensor_A,-0.040894
2,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2024-01-01,2024-01-01 00:12:35,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,370.7229,Step_0,2024-01-01 00:00:01.999999997,Sensor_A,0.005357
3,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2024-01-01,2024-01-01 00:12:35,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,370.7229,Step_1,2024-01-01 00:00:03.000000000,Sensor_A,-0.029941
4,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2024-01-01,2024-01-01 00:12:35,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,370.7229,Step_1,2024-01-01 00:00:04.000000002,Sensor_A,-0.098098
...,...,...,...,...,...,...,...,...,...,...
10825,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2024-01-01,2024-01-01 00:12:35,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,370.7229,Step_12,2024-01-01 00:11:57.000000000,Sensor_O,803.644287
10826,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2024-01-01,2024-01-01 00:12:35,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,370.7229,Step_12,2024-01-01 00:11:58.000000002,Sensor_O,815.216125
10827,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2024-01-01,2024-01-01 00:12:35,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,370.7229,Step_12,2024-01-01 00:11:58.999999998,Sensor_O,808.381531
10828,8060e8e1-504a-5138-a9f0-e2770bd61ba1,2024-01-01,2024-01-01 00:12:35,6515456b-05d4-5e6d-b42a-7652a8d671a4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,370.7229,Step_12,2024-01-01 00:12:00.000000000,Sensor_O,809.344849


In [19]:
tmp_list = []

temp_grps = [(x[0], x[1], y) for x, y in run_df_filtered.groupby(by=["Step ID", "Sensor Name"])]
for temp_grp in temp_grps:
    temp_step_id = temp_grp[0]
    temp_sensor_name = temp_grp[1]
    temp_df = temp_grp[2]
    temp_df = temp_df.reset_index()

    temp_tool_id = temp_df.loc[0, "Tool ID"]
    temp_run_start_time = temp_df.loc[0, "Run Start Time"]
    temp_run_end_time = temp_df.loc[0, "Run End Time"]
    temp_run_id = temp_df.loc[0, "Run ID"]
    temp_process_step = temp_df.loc[0, "Process Step"]
    temp_consumable_life = temp_df.loc[0, "Consumable Life"]

    sensor_avgs = temp_df["Sensor Value"].mean()

    temp_time_taken = (temp_df["Time Stamp"].max() - temp_df["Time Stamp"].min()).total_seconds()
    temp_time_taken = int(round(temp_time_taken))
   
    tmp_list.append([
        temp_process_step,
        temp_tool_id,
        temp_run_id,
        temp_consumable_life,
        temp_run_start_time,
        temp_run_end_time,
        temp_time_taken,
        temp_step_id,
        temp_sensor_name,
        sensor_avgs
    ]) 

columns = [
    "Process Step",
    "Tool ID",
    "Run ID",
    "Consumable Life",
    "Run Start Time",
    "Run End Time",
    "Time Taken (Seconds)",
    "Step ID",
    "Sensor Name",
    "Sensor Value (Average)"
] 
   
dataframe_run_filtered = pd.DataFrame(tmp_list, columns=columns) 
del temp_grps
dataframe_run_filtered = dataframe_run_filtered.sort_values(
    by=["Step ID", "Sensor Name"], 
    key=lambda x: x.str.extract(r'_(\d+)')[0].astype(float) if x.name == "Step ID" else x
)
dataframe_run_filtered


,Process Step,Tool ID,Run ID,Consumable Life,Run Start Time,Run End Time,Time Taken (Seconds),Step ID,Sensor Name,Sensor Value (Average)
0,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,2,Step_0,Sensor_A,-0.032214
1,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,2,Step_0,Sensor_B,-86.196404
2,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,2,Step_0,Sensor_C,53.251324
3,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,2,Step_0,Sensor_D,-0.062696
4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,2,Step_0,Sensor_E,0.136355
...,...,...,...,...,...,...,...,...,...,...
70,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,25,Step_12,Sensor_K,0.009278
71,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,25,Step_12,Sensor_L,0.217697
72,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,25,Step_12,Sensor_M,904.188416
73,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,25,Step_12,Sensor_N,922.470276


In [20]:
dataframe_run_filtered["Test"] = dataframe_run_filtered["Sensor Name"] + ";" + dataframe_run_filtered["Step ID"]
dataframe_run_filtered = dataframe_run_filtered.drop(columns=["Step ID", "Sensor Name"])
dataframe_run_filtered

,Process Step,Tool ID,Run ID,Consumable Life,Run Start Time,Run End Time,Time Taken (Seconds),Sensor Value (Average),Test
0,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,2,-0.032214,Sensor_A;Step_0
1,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,2,-86.196404,Sensor_B;Step_0
2,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,2,53.251324,Sensor_C;Step_0
3,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,2,-0.062696,Sensor_D;Step_0
4,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,2,0.136355,Sensor_E;Step_0
...,...,...,...,...,...,...,...,...,...
70,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,25,0.009278,Sensor_K;Step_12
71,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,25,0.217697,Sensor_L;Step_12
72,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,25,904.188416,Sensor_M;Step_12
73,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,25,922.470276,Sensor_N;Step_12


In [ ]:
dataframe_run_pivot = dataframe_run_filtered.pivot(
    index=["Process Step", "Tool ID", "Run ID", "Consumable Life", "Run Start Time", "Run End Time"],
    columns="Test",
    values="Sensor Value (Average)"
).reset_index()

dataframe_run_pivot

Test,Process Step,Tool ID,Run ID,Consumable Life,Run Start Time,Run End Time,Sensor_A;Step_0,Sensor_A;Step_1,Sensor_A;Step_10,Sensor_A;Step_11,...,Sensor_O;Step_11,Sensor_O;Step_12,Sensor_O;Step_2,Sensor_O;Step_3,Sensor_O;Step_4,Sensor_O;Step_5,Sensor_O;Step_6,Sensor_O;Step_7,Sensor_O;Step_8,Sensor_O;Step_9
0,22c8716b-1c14-525a-8c0a-0473ec7aa99e,8060e8e1-504a-5138-a9f0-e2770bd61ba1,6515456b-05d4-5e6d-b42a-7652a8d671a4,370.7229,2024-01-01,2024-01-01 00:12:35,-0.032214,-0.005383,-0.014459,0.00385,...,1013.064026,837.154419,1117.980225,1167.940552,1233.449707,1232.551392,1231.689209,1233.460815,1232.838989,1186.535522
